In [3]:
import numpy as np 
import cv2
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score

In [2]:
with_mask = np.load('with_mask.npy')
without_mask = np.load('without_mask.npy')
with_mask = with_mask.reshape(200,50*50*3)
without_mask = without_mask.reshape(200, 50*50*3)

In [4]:
X = np.r_[with_mask,without_mask]
labels = np.zeros(X.shape[0])
labels[200:] = 1.0
names = {0: 'Mask', 1: 'No Mask'}

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X,labels, test_size = 0.25)

In [15]:
pca = PCA(n_components= 3)
x_train = pca.fit_transform(x_train)

In [16]:
svm = SVC()
svm.fit(x_train, y_train)

SVC()

In [17]:
#transform into 3 dimension
x_test = pca.transform(x_test)
y_pred = svm.predict(x_test)

In [18]:
print('Regular accuracy',accuracy_score(y_test,y_pred))
print('F1 Score',f1_score(y_test,y_pred))

Regular accuracy 0.96
F1 Score 0.9607843137254902


In [ ]:
#camera
#if have video just do cv2.VideoCapture('Videoname.mp3')
haar_data = cv2.CascadeClassifier('C:/Users/user/JupyterLab/Face-Mask-Detection-master/haarcascade_frontalface_default.xml')
capture = cv2.VideoCapture(0)
data= []
font= cv2.FONT_HERSHEY_COMPLEX
while True: 
    flag,img = capture.read()
    if flag: 
        face = haar_data.detectMultiScale(img)
        for x,y,w,h in face:
            cv2.rectangle(img, (x,y), (x+w, y+h),(255,0,255),4)
            face1 = img[y:y+h, x: x+w, :]
            face1 = cv2.resize(face1, (50,50))
            face1 = face1.reshape(1,-1)
            face1 = pca.transform(face1)
            pred = svm.predict(face1)
            n = names[int(pred)]
            cv2.putText(img,n,(x,y), font, 1,(244,250,250),4 )
            print(n)
        cv2.imshow('Mask detection',img)
        #27 ==> Escape button in ASCII
        if cv2.waitKey(2) == 27: 
            break
            
capture.release()
cv2.destroyAllWindows()